# Global Forecasting System - Meteorological forecast

In [ ]:
from datetime import datetime, timedelta
import xarray
import numpy as np
import pandas as pd
import mikeio

The file `gfs_wind.nc` contains a small sample of the [GFS](https://nomads.ncep.noaa.gov/) forecast data downloaded via their OpenDAP service

In [ ]:
ds = xarray.open_dataset('../tests/testdata/gfs_wind.nc')

In [ ]:
ds.time.values[0],ds.time.values[-1]

In [ ]:
(ds.time.values[-1]- ds.time.values[0]).astype('timedelta64[D]')

The forecast contains data for the coming 10 days.

Running a Mike 21 HD model, needs at least three variables of meteorlogical forcing
* Mean Sea Level Pressure
* U 10m
* V 10m

In [ ]:
ds.msletmsl

In [ ]:
ds.ugrd10m

In [ ]:
ds.vgrd10m

In [ ]:
ds.ugrd10m[0].plot()

## Convert to dfs2

## Time

In [ ]:
time = pd.DatetimeIndex(ds.time)
time

## Variable types

In [ ]:
mikeio.EUMType.Air_Pressure

In [ ]:
mikeio.EUMType.Air_Pressure.units

In [ ]:
mikeio.EUMType.Wind_Velocity

In [ ]:
mikeio.EUMType.Wind_Velocity.units

In [ ]:
mslp = ds.msletmsl.values / 100 # conversion from Pa to hPa
u = ds.ugrd10m.values
v = ds.vgrd10m.values

In [ ]:
geometry = mikeio.Grid2D(x=ds.lon.values, y=ds.lat.values, projection="LONG/LAT")
geometry

In [ ]:
from mikeio import ItemInfo, EUMType, EUMUnit

mslp_da = mikeio.DataArray(data=mslp,time=time, geometry=geometry, item=ItemInfo("Mean Sea Level Pressure", EUMType.Air_Pressure, EUMUnit.hectopascal))
u_da = mikeio.DataArray(data=u,time=time, geometry=geometry, item=ItemInfo("Wind U", EUMType.Wind_Velocity, EUMUnit.meter_per_sec))
v_da = mikeio.DataArray(data=v,time=time, geometry=geometry, item=ItemInfo("Wind V", EUMType.Wind_Velocity, EUMUnit.meter_per_sec))

In [ ]:
mds = mikeio.Dataset([mslp_da, u_da, v_da])
mds

In [ ]:
mds.to_dfs("gfs.dfs2")

## Clean up (don't run this)

In [ ]:
import os
os.remove("gfs.dfs2")